# 📚 Hệ gợi ý Memory-based: Item-based CF (Cosine + Mean-centering + Shrinkage)

Notebook này hiện thực **Item-based Collaborative Filtering** theo hướng memory-based:
- Chuẩn hoá ID → chỉ số 0..n-1 (gộp cả train & test)
- Ma trận rating thưa **R (user × item)**
- **Mean-centering** theo từng user: $\tilde{r}_{ui} = r_{ui} - \mu_u$
- **Cosine similarity** giữa cột item trên ma trận đã center
- **Shrinkage** theo số user đồng-rating: $s' = \frac{n_{co}}{n_{co}+\alpha} s$ trong đó $n_{co}$ là số người dùng đã đánh giá cả 2 item i và j
- Dự đoán: $\hat{r}_{ui} = \mu_u + \frac{\sum_j s'(i,j)\,(r_{uj}-\mu_u)}{\sum_j |s'(i,j)|}$ (top-K láng giềng)
- Xuất file **submission** dạng `Id,Score` (Id = 1..N theo thứ tự test)


In [1]:

import numpy as np
import pandas as pd
from scipy import sparse
from pathlib import Path

# --- Đường dẫn (sửa nếu cần) ---
TRAIN_PATH = "data/train.txt"   # ví dụ: "/kaggle/input/.../train.txt"
TEST_PATH  = "data/test.txt"    # ví dụ: "/kaggle/input/.../test.txt"
SUB_PATH   = "output/submission_itemCF.csv"

print(Path(TRAIN_PATH).resolve())
print(Path(TEST_PATH).resolve())


E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\train.txt
E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\test.txt


In [2]:

# --- Đọc dữ liệu cách bởi khoảng trắng/tab ---
df_train = pd.read_csv(
    TRAIN_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid","rating"],
    engine="python",
)
df_test = pd.read_csv(
    TEST_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid"],
    engine="python",
)
print(df_train.shape, df_test.shape)
display(df_train.head())
display(df_test.head())


(90570, 3) (9430, 2)


,userid,movieid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


,userid,movieid
0,1,20
1,1,33
2,1,61
3,1,117
4,1,155


In [3]:

# --- Ánh xạ ID gốc -> chỉ số liên tục 0..n-1 (gộp cả train & test để không thiếu id) ---
all_u = pd.concat([df_train["userid"], df_test["userid"]], ignore_index=True)
all_i = pd.concat([df_train["movieid"], df_test["movieid"]], ignore_index=True)

uid_uniques = all_u.drop_duplicates()
iid_uniques = all_i.drop_duplicates()

uid2idx = pd.Series(np.arange(len(uid_uniques), dtype=np.int32), index=uid_uniques.values)
iid2idx = pd.Series(np.arange(len(iid_uniques), dtype=np.int32), index=iid_uniques.values)

df_train["u_idx"] = df_train["userid"].map(uid2idx).astype(np.int32)
df_train["i_idx"] = df_train["movieid"].map(iid2idx).astype(np.int32)
df_test["u_idx"]  = df_test["userid"].map(uid2idx).astype(np.int32)
df_test["i_idx"]  = df_test["movieid"].map(iid2idx).astype(np.int32)

n_users = uid2idx.size
n_items = iid2idx.size
print(f"n_users={n_users}, n_items={n_items}, train_rows={len(df_train)}, test_rows={len(df_test)}")
df_train

n_users=943, n_items=1682, train_rows=90570, test_rows=9430


,userid,movieid,rating,u_idx,i_idx
0,1,1,5,0,0
1,1,2,3,0,1
2,1,3,4,0,2
3,1,4,3,0,3
4,1,5,3,0,4
...,...,...,...,...,...
90565,943,1047,2,942,1034
90566,943,1074,4,942,1063
90567,943,1188,3,942,1173
90568,943,1228,3,942,1211


In [4]:

# --- Tạo ma trận thưa R (user × item) và mean-center theo từng user ---
rows = df_train["u_idx"].to_numpy() # lấy cột chỉ số người dùng (user index) thành mảng NumPy. Mỗi phần tử là một hàng trong ma trận.
cols = df_train["i_idx"].to_numpy() # lấy cột chỉ số vật phẩm (item index) thành mảng NumPy. Mỗi phần tử là một cột trong ma trận.

vals = df_train["rating"].astype(np.float32).to_numpy() 
# ấy cột điểm đánh giá, ép kiểu sang float32 (giảm bộ nhớ) rồi thành mảng NumPy. Đây là các giá trị ở vị trí (row, col) tương ứng

R = sparse.csr_matrix((vals, (rows, cols)), shape=(n_users, n_items)) # CSR (Compressed Sparse Row) kích thước n_users × n_items
# ở mỗi cặp tọa độ (rows[k], cols[k]) đặt giá trị vals[k]. Các phần tử còn lại mặc định là 0

print(R)

# Trung bình theo user (mu_u). Nếu user chưa có rating -> dùng global mean
# Tổng rating theo từng user (theo hàng)
user_sum = np.array(R.sum(axis=1)).ravel() # np.array(...).ravel() chuyển về mảng 1D dạng (n_users,)
user_cnt = np.diff(R.indptr)  # số phần tử khác 0 mỗi dòng

# Trung bình toàn cục của tất cả rating trong tập train dùng làm fallback nếu user chưa có rating
global_mean = float(vals.mean()) if len(vals) else 3.0

user_mean = np.divide(
    user_sum,
    user_cnt,
    out=np.full_like(user_sum, global_mean, dtype=np.float64),
    where=user_cnt != 0
)

# user_mean là mảng (n_users,), mỗi phần tử là:
# - trung bình rating của user đó, nếu user có rating;
# - global_mean, nếu user chưa có rating.

# R_centered = R - mu_u theo từng dòng
R_centered = R.tocsr(copy=True).astype(np.float32) # Sao chép R sang CSR mới để không sửa ma trận gốc.

for u in range(n_users):
    s, e = R_centered.indptr[u], R_centered.indptr[u+1]
    if s < e:
        R_centered.data[s:e] -= user_mean[u]

# Chuyển sang CSC để thao tác theo cột (item) nhanh
X = R_centered.tocsc()
print("Matrix ready:", R.shape, "nnz:", R.nnz)


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 90570 stored elements and shape (943, 1682)>
  Coords	Values
  (0, 0)	5.0
  (0, 1)	3.0
  (0, 2)	4.0
  (0, 3)	3.0
  (0, 4)	3.0
  (0, 5)	5.0
  (0, 6)	4.0
  (0, 7)	1.0
  (0, 8)	5.0
  (0, 9)	3.0
  (0, 10)	2.0
  (0, 11)	5.0
  (0, 12)	5.0
  (0, 13)	5.0
  (0, 14)	5.0
  (0, 15)	5.0
  (0, 16)	3.0
  (0, 17)	4.0
  (0, 18)	5.0
  (0, 19)	1.0
  (0, 20)	4.0
  (0, 21)	4.0
  (0, 22)	3.0
  (0, 23)	4.0
  (0, 24)	3.0
  :	:
  (942, 703)	1.0
  (942, 716)	4.0
  (942, 744)	4.0
  (942, 747)	2.0
  (942, 754)	4.0
  (942, 756)	3.0
  (942, 776)	2.0
  (942, 785)	3.0
  (942, 787)	3.0
  (942, 807)	4.0
  (942, 815)	4.0
  (942, 816)	3.0
  (942, 822)	2.0
  (942, 830)	4.0
  (942, 919)	5.0
  (942, 931)	1.0
  (942, 933)	5.0
  (942, 1000)	2.0
  (942, 1014)	2.0
  (942, 1031)	3.0
  (942, 1034)	2.0
  (942, 1063)	4.0
  (942, 1173)	3.0
  (942, 1211)	3.0
  (942, 1322)	3.0
Matrix ready: (943, 1682) nnz: 90570


# Giải thích toán học cho hàm `predict_ui_item_based`

Hàm dự đoán theo **Item-based CF** với các bước: **mean-centering theo user → cosine similarity → shrinkage → chọn top-K → nội suy phần dư → fallback + clip**.

---

## Ký hiệu

- $r_{u,i}$: rating thật (đã quan sát) của user $u$ cho item $i$.
- $\mu_u$: trung bình rating của user $u$ (fallback = global mean nếu $u$ chưa có rating).
- Ma trận “trừ trung bình theo user”:
  $$
  x_{u,i} =
  \begin{cases}
    r_{u,i} - \mu_u, & \text{nếu } (u,i) \text{ có rating} \\
    0, & \text{ngược lại}
  \end{cases}
  $$
  Gọi $\mathbf{x}_i \in \mathbb{R}^{U}$ là **vector cột** của item $i$.

- $J(u)$: tập item mà user $u$ đã rating.

---

## Bước 1 — Cosine similarity

Với item đích $i$ và mỗi $j \in J(u)$:
$$
s_{i,j} \;=\; \cos(\mathbf{x}_i,\mathbf{x}_j)
\;=\; \frac{\mathbf{x}_i^\top \mathbf{x}_j}{\|\mathbf{x}_i\|_2 \,\|\mathbf{x}_j\|_2}.
$$
(Trong code: `num = (xi.T @ XJ)`, `denom = xi_norm * XJ_norm`, `sim = num / denom`.)

---

## Bước 2 — Shrinkage theo số user đồng-xếp hạng

Gọi $n_{i,j}$ là số user đồng thời có rating cho $i$ và $j$. Với tham số $\alpha > 0$:
$$
\tilde{s}_{i,j} \;=\; \frac{n_{i,j}}{n_{i,j}+\alpha}\, s_{i,j}.
$$
(Trong code: `n_co = (xi_bin.T @ XJ_bin)`, `sim = sim * (n_co/(n_co+alpha))`.)

---

## Bước 3 — Chọn láng giềng top-K

Chọn
$$
\mathcal{N}_K(u,i) = \operatorname*{arg\,topK}_{j \in J(u)} \bigl| \tilde{s}_{i,j} \bigr|.
$$
(Trong code: `argpartition` chọn nhanh top-K theo ∣sim∣.)

---

## Bước 4 — Suy luận rating bằng trung bình có trọng số (trên **phần dư**)

$$
\hat{r}_{u,i}
\;=\;
\mu_u
\;+\;
\frac{\displaystyle \sum_{j \in \mathcal{N}_K(u,i)} \tilde{s}_{i,j}\,\bigl(r_{u,j} - \mu_u\bigr)}
{\displaystyle \sum_{j \in \mathcal{N}_K(u,i)} \bigl|\tilde{s}_{i,j}\bigr| }.
$$

(Trong code: `r_u_centered = R_centered[u, J_top]`,  
`delta = sum(sim_top * r_u_centered) / sum(|sim_top|)`,  
`r_hat = user_mean[u] + delta`.)

---

## Bước 5 — Fallback & Clipping

- Nếu $J(u) = \varnothing$, hoặc cột $\mathbf{x}_i$ rỗng, hoặc $\sum |\tilde{s}_{i,j}| \approx 0$  
  $\Rightarrow$ **fallback**: $\hat{r}_{u,i} \leftarrow \mu_u$ (hoặc global mean).
- **Kẹp** về miền hợp lệ, ví dụ $[1,5]$:
$$
\hat{r}_{u,i} \leftarrow \min\bigl(\max(\hat{r}_{u,i},\,1),\,5\bigr).
$$

---

## Tóm tắt

Hàm thực hiện **Item-based CF** trên ma trận **đã trừ trung bình theo user**, đo tương tự **cosine**, áp dụng **shrinkage** để giảm nhiễu khi giao nhau ít, **chọn top-K** láng giềng có $|\tilde{s}|$ lớn, rồi **nội suy phần dư** và cộng lại $\mu_u$. Cách này khử thiên lệch “dễ/khó tính” theo user và thường cho tương tự ổn định hơn.


In [5]:
def predict_ui_item_based(u, i, K=50, alpha=20.0, clip_min=1.0, clip_max=5.0):
    # Các item J mà user u đã có rating
    s, e = R.indptr[u], R.indptr[u+1]
    J = R.indices[s:e]
    if J.size == 0:
        # fallback: trung bình người dùng (hoặc global_mean nếu cần)
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    # Cột i (đã center) và các cột tương ứng item J
    xi = X[:, i]
    if xi.nnz == 0:
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))
    XJ = X[:, J]

    # Cosine similarity (chuyển kết quả về mảng 1D)
    num = (xi.T @ XJ).toarray().ravel()
    xi_norm = np.sqrt(xi.multiply(xi).sum())
    XJ_norm = np.sqrt(np.array((XJ.multiply(XJ)).sum(axis=0)).ravel())
    denom = xi_norm * XJ_norm
    sim = np.divide(num, denom, out=np.zeros_like(num), where=denom != 0)

    # Shrinkage: s' = n_co/(n_co+alpha) * s  (chia an toàn)
    xi_bin = (xi != 0).astype(np.int8)
    XJ_bin = (XJ != 0).astype(np.int8)
    n_co = (xi_bin.T @ XJ_bin).toarray().ravel()
    shrink = np.divide(n_co, n_co + alpha, out=np.zeros_like(n_co, dtype=float), where=(n_co + alpha) != 0)
    sim = sim * shrink

    # Top-K theo |sim|
    if J.size > K:
        topk_idx = np.argpartition(np.abs(sim), -K)[-K:]
        J_top   = J[topk_idx]
        sim_top = sim[topk_idx]
    else:
        J_top   = J
        sim_top = sim

    # (r_uj - μ_u) và công thức suy luận
    r_u_centered = R_centered[u, J_top].toarray().ravel()
    denom2 = float(np.sum(np.abs(sim_top)))

    # Nếu không có hàng xóm hợp lệ -> fallback
    if not np.isfinite(denom2) or denom2 < 1e-12:
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    delta = float(np.sum(sim_top * r_u_centered) / denom2)
    if not np.isfinite(delta):
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    r_hat = user_mean[u] + delta
    return float(np.clip(r_hat, clip_min, clip_max))


In [6]:
K = 50
ALPHA = 20.0

preds = np.array(
    [predict_ui_item_based(u, i, K=K, alpha=ALPHA)
     for u, i in zip(df_test["u_idx"].to_numpy(), df_test["i_idx"].to_numpy())],
    dtype=np.float64
)

# Đảm bảo mọi giá trị đều hữu hạn (không NaN/Inf)
finite_mask = np.isfinite(preds)
if not finite_mask.all():
    safe_mean = np.nanmean(preds[finite_mask]) if finite_mask.any() else float(global_mean)
    preds[~finite_mask] = safe_mean

# (tùy) clip về [1,5]
preds = np.clip(preds, 1.0, 5.0)

submission = pd.DataFrame({
    "Id": np.arange(1, len(preds) + 1, dtype=np.int64),
    "Score": preds
})

# Ghi file — dùng float_format để tránh ký tự rỗng kỳ quặc
submission.to_csv(SUB_PATH, index=False, float_format="%.6f")
print("Đã lưu:", SUB_PATH, submission.shape, submission.dtypes)


Đã lưu: output/submission_itemCF.csv (9430, 2) Id         int64
Score    float64
dtype: object
